In [1]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer

import pandas as pd

In [2]:
import transformers
print(transformers.__version__)

embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2", device= "cpu")

4.40.1


In [3]:
data = pd.read_csv('bdd.csv')
data

,page_number,sentence_chunk,tokens_number
0,1,1 Préambule Après onze années en tant que ch...,94.00
1,1,J’ai alors fait des diapos pour chaque chapitr...,46.75
2,2,"2 Pour finir, je pense avoir synthétisé ma mo...",12.75
3,13,13 Présentation du cours Ce cours s’adresse ...,77.50
4,13,"Théorie des ensembles, 3. Logique du premier ...",19.50
...,...,...,...
160,145,145 Annexe 1 : Les Fonctions dans SQL 92 Do...,69.25
161,146,146 IS [NOT] NULL NULL INNER JOIN Jointure...,28.50
162,147,147 Annexe 2 : Outils Pédagogiques 1. Functi...,45.25
163,147,Il est utilisé par les étudiants pour confront...,12.75


In [4]:
list_chunk = [item for item in data['sentence_chunk']]
len(list_chunk)

165

In [5]:
embedding_model.to("cpu")

SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [6]:
%%time

#embed all texts in batchs of 32 
text_chunk_embeddings = embedding_model.encode(sentences=list_chunk,
                                               batch_size = 32,
                                               convert_to_tensor=True )
text_chunk_embeddings

CPU times: total: 3min 7s
Wall time: 1min 45s


tensor([[ 0.0084, -0.0511, -0.0165,  ...,  0.0472, -0.0016, -0.0583],
        [-0.0048, -0.0710, -0.0338,  ...,  0.0559, -0.0208, -0.0427],
        [ 0.0315,  0.0174, -0.0177,  ...,  0.0601,  0.0472, -0.0627],
        ...,
        [-0.0117,  0.0014, -0.0473,  ..., -0.0003, -0.0136, -0.0288],
        [-0.0045, -0.0186, -0.0273,  ...,  0.0312, -0.0201, -0.0280],
        [-0.0380, -0.0087, -0.0266,  ...,  0.0166, -0.0234, -0.0320]])

In [7]:
text_chunk_embeddings.shape

torch.Size([165, 768])

In [8]:
#save the embeddings 
embedding_values = text_chunk_embeddings.tolist()
data['embeddings'] = embedding_values
data.head()

data.to_csv("bdd.csv", index=False)